In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
%matplotlib qt5
torch.set_num_interop_threads(8)
torch.set_num_threads(8)

QFont::fromString: Invalid description 'Noto Sans,11,-1,5,700,0,0,0,0,0,0,0,0,0,0,1,Bold'
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,700,0,0,0,0,0,0,0,0,0,0,1,Bold'
QFont::fromString: Invalid description 'Noto Sans,11,-1,5,700,0,0,0,0,0,0,0,0,0,0,1,Bold'


In [ ]:
import pandas as pd
import torch
import pickle

df = pd.read_csv('/home/user456/CommonRuntime/python ML/train.csv')
df = df.dropna()
df = df.drop(columns=['fnlwgt','education','capital-gain','capital-loss'])

cat_cols = [
    'workclass','marital-status','occupation',
    'relationship','race','gender','native-country'
]

mappings = {}  # to store categories

for col in cat_cols:
    codes, uniques = pd.factorize(df[col])
    df[col + '_id'] = codes
    mappings[col] = list(uniques)

df = df.drop(columns=cat_cols)

print(df.head())
print(df.columns)
y = torch.tensor(df['occupation_id'].values, dtype=torch.int64)
df = df.drop(columns=['occupation_id', 'income_>50K'])

x = torch.tensor(df.values, dtype=torch.float32)

print(y.shape, x.shape)

# persist mapping
with open("cat_mappings.pkl", "wb") as f:
    pickle.dump(mappings, f)

print("Mapping saved!")


   age  educational-num  hours-per-week  income_>50K  workclass_id  \
0   67               16              60            1             0   
1   17                8              15            0             0   
2   31               13              40            1             0   
3   58                4              40            0             1   
4   25               10              40            0             1   

   marital-status_id  occupation_id  relationship_id  race_id  gender_id  \
0                  0              0                0        0          0   
1                  1              1                1        0          0   
2                  2              0                2        0          0   
3                  2              2                2        0          0   
4                  1              1                0        1          0   

   native-country_id  
0                  0  
1                  0  
2                  0  
3                  0  
4      

In [53]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
x=x.to(device)
y=y.to(device)

class CEModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(9, 256)
        self.layer2 = nn.ReLU()
        
        self.layer3 = nn.Linear(256, 128)
        self.layer4 = nn.ReLU()
        self.drop = nn.Dropout(0)

        self.layer5 = nn.Linear(128, 64)
        self.layer6 = nn.ReLU()

        self.layer7 = nn.Linear(64, 14)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)

        out = self.layer3(out)
        out = self.layer4(out)
        out = self.drop(out)

        out = self.layer5(out)
        out = self.layer6(out)

        out = self.layer7(out)
        return out

lossy=nn.CrossEntropyLoss()
model=CEModel()
fig,axes=plt.subplots()
g=axes.plot([0],[0],color="blue")

Using device: cpu


In [ ]:
optimizer=optim.AdamW(model.parameters(),lr=1e-3)

In [ ]:
for i in range(20000):
    y_pred=model(x.float())
    lossval=lossy(y_pred,y)
    optimizer.zero_grad()
    lossval.backward()
    optimizer.step()
    print(f'{i+1} epoch with loss {lossval.item()}')
    if (i+1)%100==0:
        print(f'{i+1} epoch with loss {lossval.item()}')
        losss.append(lossval.item())
        g[0].set_xdata(range(len(losss)))
        g[0].set_ydata(losss)
        axes.relim()
        axes.autoscale_view()
        plt.pause(0.01)
plt.show()
with torch.no_grad():
    y_pred=model(x)
    predicted_cls=torch.argmax(y_pred,dim=1)
    accuracy=(predicted_cls==y).sum().item()/len(y)
    print(f'Accuracy is {accuracy}%')

In [ ]:
with open("cat_mappings.pkl", "rb") as f:
    mappings = pickle.load(f)

test_df = pd.read_csv("/home/user456/CommonRuntime/python ML/__pycache__/test.csv")

cat_cols = list(mappings.keys())

for col in cat_cols:
    mapping = {v:i for i,v in enumerate(mappings[col])}

    test_df[col + "_id"] = test_df[col].map(mapping).fillna(-1).astype(int)

test_df = test_df.drop(columns=cat_cols)
test_df=test_df.drop(columns=['fnlwgt','education','capital-gain','capital-loss'])
test_df
# Target and features
y = torch.tensor(test_df['occupation_id'].values, dtype=torch.int64)
test_df = test_df.drop(columns=['occupation_id'])

x = torch.tensor(test_df.values, dtype=torch.float32)

print(y.shape, x.shape)

torch.Size([899]) torch.Size([899, 9])


In [ ]:
import joblib

z = joblib.load(
    '/home/user456/CommonRuntime/python ML/job_model2.pkl',  
)

model.load_state_dict(z['model'])
print(model)


CEModel(
  (layer1): Linear(in_features=9, out_features=256, bias=True)
  (layer2): ReLU()
  (layer3): Linear(in_features=256, out_features=128, bias=True)
  (layer4): ReLU()
  (drop): Dropout(p=0, inplace=False)
  (layer5): Linear(in_features=128, out_features=64, bias=True)
  (layer6): ReLU()
  (layer7): Linear(in_features=64, out_features=14, bias=True)
)


In [ ]:
import pandas as pd
import pickle
with open("cat_mappings.pkl", "rb") as f:
    mappings = pickle.load(f)

occ_list = mappings["occupation"]              
occ_map_df = pd.DataFrame({
    "occupation_id": range(len(occ_list)),
    "occupation_name": occ_list
})
real_ids = y.cpu().numpy()
pred_ids = predicted_cls.cpu().numpy()

real_names = [occ_list[i] if i < len(occ_list) else "Unknown" for i in real_ids]
pred_names = [occ_list[i] if i < len(occ_list) else "Unknown" for i in pred_ids]

compare_df = pd.DataFrame({
    "real_occ_id": real_ids,
    "real_occ_name": real_names,
    "pred_occ_id": pred_ids,
    "pred_occ_name": pred_names,
})

accuracy = (predicted_cls == y).sum().item() / len(y)
print(f"Accuracy: {accuracy*100:.2f}%")
output_path = "pred_vs_real.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
    occ_map_df.to_excel(writer, sheet_name="occupation_mapping", index=False)
    compare_df.to_excel(writer, sheet_name="pred_vs_real", index=False)

print("File saved:", output_path)


Accuracy: 28.59%
File saved: pred_vs_real.xlsx
File saved: pred_vs_real.xlsx


Sum: 5000000050000000
Time taken: 11.73875617980957
